In [1]:
import numpy as np
import math
from enum import Enum
from collections import namedtuple

# matplotlib
# ライブラリーのインポート
import matplotlib.pyplot as plt
# インライン表示
%matplotlib inline

# 日本語フォント
import pylab
from matplotlib.font_manager import FontProperties
fp = FontProperties(fname='C:\WINDOWS\Fonts\msgothic.ttc', size = 12)

# 第九章　自然エネルギー利用設備

## 第三節　空気集熱式太陽熱利用設備

### 5. 給湯部における補正集熱量

日付$d$における1日当たりの給湯部における補正集熱量$L_{sun,ass,d}$

$$
\begin{equation*}
    L_{sun,acc,d} = min \left( L_{tunk,d} , \: \left( L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} \right) \times f_{sr,uplim} \right)
\end{equation*}
$$

$f_{sr,uplim}$ : 給湯部の分担率上限値(-)   
$L_{sun,acc,d}$ : 日付$d$における1日当たりの給湯部における補正集熱量(MJ/日)  
$L_{tnk,d}$ : 日付$d$における1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量(MJ/日)  
$L'_{k,d}$ : 日付$d$における1日当たりの台所水栓における節湯補正給湯熱負荷(MJ/日)  
$L'_{s,d}$ : 日付$d$における1日当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/日)  
$L'_{w,d}$ : 日付$d$における1日当たりの洗面水栓における節湯補正給湯熱負荷(MJ/日)  
$L'_{b1,d}$ : 日付$d$における1日当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/日)  
$L'_{b2,d}$ : 日付$d$における1日当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/日)  
$L'_{ba1,d}$ : 日付$d$における1日当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/日)

給湯部の分担率上限値$f_{sr,uplim}$

$$
\begin{equation*}
    f_{sr,uplim} = 0.9
\end{equation*}
$$

日付$d$における1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量$L_{tnk,d}$

$$
\begin{equation*}
   L_{tnk,d} = min \left( Q_{d} , \: HC_{tnk,d} \times \alpha_{tnk,d} \right)
\end{equation*}
$$

$L_{tnk,d}$ : 日付$d$における1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量(MJ/日)  
$Q_{d}$ : 日付$d$における1日当たりの基準集熱量(MJ/日)  
$HC_{tnk,d}$ : 日付$d$における給湯部のタンク蓄熱量の上限(MJ)  
$\alpha_{tnk,d}$ : 日付$d$における給湯部のタンク有効利用率(1/日)  

日付$d$における給湯部のタンク有効利用率$\alpha_{tnk,d}$

$$
\begin{equation*}
    \alpha_{tnk,d} = 1.0
\end{equation*}
$$

In [2]:
def get_L_tnk(Q_d, HC_tnk):
    alpha_tnk = 1  # 1/day
    return min(Q_d, HC_tnk * alpha_tnk)

日付$d$における給湯部のタンク蓄熱量の上限$HC_{tnk,d}$

$$
\begin{equation*}
    HC_{tnk,d} = \left( \theta_{tnk} - \theta_{wtr,d} \right) \times W_{tnk} \times 10^{-3}
\end{equation*}
$$

$HC_{tnk,d}$ : 日付$d$における給湯部のタンク蓄熱量の上限(MJ)  
$\theta_{tnk}$ : 給湯部のタンク内温度(℃)   
$\theta_{wtr,d}$ : 日付$d$における日平均給水温度(℃)  
$W_{tnk}$ : 給湯部のタンク容量(L)  

給湯部のタンク内温度$\theta_{tnk}$

$$
\begin{equation*}
    \theta_{tnk} = 65
\end{equation*}
$$

In [3]:
def get_HC_tnk(theta_wtr, W_tnk):
    theta_tnk = 65  # degree C
    return (theta_tnk - theta_wtr) * W_tnk * 10**(-3)

### 6. 基準集熱量

日付$d$における1日当たりの基準集熱量$Q_{d}$

$$
\begin{equation*}
    Q_{d} = \sum_{t=0}^{23} \left( Q_{col,d,t} \times f_{hx} \times f_{s} \times t_{cp,d,t} \right)
\end{equation*}
$$

$f_{hx}$ : 給湯部の熱交換効率(-)  
$f_{s}$ : 給湯部のシステム効率(-)  
$Q_{d}$ : 日付$d$における1日当たりの基準集熱量(MJ/日)  
$Q_{col,d,t}$ : 日付$d$の時刻$t$における1時間当たりの集熱部における集熱量(MJ/h)   
$t_{cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間(h/h)   

給湯部の熱交換効率$f_{hx}$

$$
\begin{equation*}
    f_{hx} = 0.25
\end{equation*}
$$

給湯部のシステム効率$f_{s}$

$$
\begin{equation*}
    f_{s} = 0.85
\end{equation*}
$$

In [4]:
def get_Q_d(Q_col, t_cp):    
    f_hx = 0.25  # -
    f_s = 0.85   # -
    return sum(Q_col[t] * f_hx * f_s * t_cp[t] for t in range(0,24,1))

### 7. 消費電力量

#### 7.1　補機の消費電力量

日付$d$における1日当たりの補機の消費電力量のうち給湯設備への付加分$E_{E,ass,aux,W,d}$

$$
\begin{equation*}
    E_{E,ass,aux,W,d} = \sum_{t=0}^{23} E_{E,ass,aux,W,d,t}
\end{equation*}
$$

$E_{E,ass,aux,W,d}$ : 日付$d$における1日当たりの補機の消費電力量のうち給湯設備への付加分(kWh/日)   
$E_{E,ass,aux,W,d,t}$ : 日付$d$の時刻$t$における1時間当たりの補機の消費電力量のうち給湯設備への付加分(kWh/h)   

日付$d$の時刻$t$における1時間当たりの補機の消費電力量のうち給湯設備への付加分$E_{E,ass,aux,W,d,t}$

$$
\begin{equation*}
    E_{E,ass,aux,W,d,t} = E_{E,fan,d,t} + E_{E,cp,d,t}
\end{equation*}
$$

$E_{E,ass,aux,W,d,t}$ : 日付$d$の時刻$t$における1時間当たりの補機の消費電力量のうち給湯設備への付加分(kWh/h)   
$E_{E,cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの消費電力量(kWh/h)   
$E_{E,fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの消費電力量(kWh/h)   

In [5]:
def get_E_E_ass_aux_W(E_E_fan, E_E_cp):    
    return sum(E_E_fan[t] + E_E_cp[t] for t in range(0,24,1))

#### 7.2　空気搬送ファンの消費電力量

日付$d$における1日当たりの空気搬送ファンの消費電力量$E_{E,fan,d,t}$

<p style="text-indent:2em">空気搬送ファンの自立運転用太陽光発電装置を採用する場合：</p>  
$$ 
\begin{equation*}
    E_{E,fan,d,t} =0
\end{equation*}
$$

<p style="text-indent:2em">空気搬送ファンの自立運転用太陽光発電装置を採用しない場合：</p> 
<p style="text-indent:4em">$t_{cp,d,t} = 0$の場合（循環ポンプが稼働していない場合）：</p> 
$$ 
\begin{equation*}
    E_{E,fan,d,t} = 0
\end{equation*}
$$

<p style="text-indent:4em">$0 < t_{cp,d,t}$の場合（循環ポンプが稼働している場合）：</p> 
$$ 
\begin{equation*}
    E_{E,fan,d,t} =f_{SFP} \times V_{fan,d,t} \times 10^{-3}
\end{equation*}
$$

$E_{E,fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの消費電力量(kWh/h)  
$f_{SFP}$ : 空気搬送ファンの比消費電力(W/(m<sup>3</sup>/h))  
$t_{cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間(h/h)  
$V_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの風量(m<sup>3</sup>/h)  

空気搬送ファンの比消費電力$f_{SFP}$

<table>
    <caption>表3 空気搬送ファンの比消費電力$f_{SFP}$</caption>
    <thead style="background-color:lightgrey" >
        <tr>
            <th style="text-align:center; text-align:center; border:1px solid black;">ファンの種別</th>
            <th style="text-align:center; text-align:center; border:1px solid black;">ファンの比消費電力$f_{SFP}$<br/>(W/(m<sup>3</sup>/h))</th>
        </tr>
    </thead>
    <tbody>    
        <tr>
            <td style="border:1px solid black; text-align:center;">ACファン</td>
            <td style="border:1px solid black; text-align:center;">0.4</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">DCファン</td>
            <td style="border:1px solid black; text-align:center;">0.2</td>
        </tr>
    </tbody> 
<table>

In [6]:
def get_f_SFP(Fan_Type):
    return {'AC' : 0.4, 'DC' : 0.2}[Fan_Type]

In [7]:
def get_E_E_fan(PV_for_Fan, Fan_Type, t_cp, V_fan):
    if PV_for_Fan is False:
        return 0
    elif PV_for_Fan is True:
        if t_cp == 0:
            return 0
        else:
            f_SFP = get_f_SFP(Fan_Type)  # W/(m3/h)
            return f_SFP * V_fan * 10**(-3)
    else:
        raise Exeption

#### 7.3　循環ポンプの消費電力量

日付$d$の時刻$t$における1時間当たりの循環ポンプの消費電力量$E_{E,cp,d,t}$

$$
\begin{equation*}
    E_{E,cp,d,t} = P_{cp} \times t_{cp,d,t} \times 10^{-3}
\end{equation*}
$$

$E_{E,cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの消費電力量(kWh/h)   
$P_{cp}$ : 循環ポンプの消費電力(W)   
$t_{cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間(h/h)  

循環ポンプの消費電力$P_{cp}$

<table>
    <caption>表4 循環ポンプの消費電力$P_{cp}$</caption>
    <thead style="background-color:lightgrey" >
        <tr>
            <th style="text-align:center; text-align:center; border:1px solid black;">循環ポンプの自立運転用太陽光発電装置の採用の有無</th>
            <th style="text-align:center; text-align:center; border:1px solid black;">循環ポンプの消費電力$P_{cp}$<br/>(W)</th>
        </tr>
    </thead>
    <tbody>    
        <tr>
            <td style="border:1px solid black; text-align:center;">採用しない</td>
            <td style="border:1px solid black; text-align:center;">80</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">採用する</td>
            <td style="border:1px solid black; text-align:center;">0</td>
        </tr>
    </tbody> 
<table>

In [8]:
def get_P_cp(PV_for_Pump):
    return {False : 80, True : 0}[PV_for_Pump]

In [9]:
def get_E_E_pump(PV_for_Pump, t_cp):
    P_cp = get_P_cp(PV_for_Pump)  # W
    return P_cp * t_cp * 10**(-3)

### 8. 空気搬送ファンおよび循環ポンプ

#### 8.1　空気搬送ファンの稼働時間

日付$d$の時刻$t$における1時間当たりの空気搬送ファンの稼働時間$t_{fan,d,t}$  

<p style="text-indent:2em">$30 \leq \theta_{col,nonopg,d,t}$ かつ $25 < \theta_{col,opg,d,t}$の場合：</p> 
$$ 
\begin{equation*}
   t_{fan,d,t} = 1
\end{equation*}
$$

<p style="text-indent:2em">$\theta_{col,nonopg,d,t} < 30$ かつ $\theta_{col,opg,d,t} \leq 25$の場合：</p> 
$$ 
\begin{equation*}
   t_{fan,d,t} = 0
\end{equation*}
$$

$t_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの稼働時間(h/h)  
$\theta_{col,opg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度(℃)  
$\theta_{col,nonopg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱部の出口における空気温度(℃)  

In [10]:
def get_t_fan(theta_col_opg, theta_col_nonopg):
    if 30 <= theta_col_nonopg and 25 < theta_col_opg:
        return 1
    elif theta_col_nonopg < 30 and theta_col_opg <= 25:
        return 0
    else:
        raise Exeption

#### 8.2　循環ポンプの稼働時間

日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間$t_{cp,d,t}$

<p style="text-indent:2em">$0 < t_{fan,d,t}$の場合：</p> 
$$ 
\begin{equation*}
   t_{cp,d,t} = 1 \times r_{cp}
\end{equation*}
$$

<p style="text-indent:2em">$t_{fan,d,t} = 0$の場合：</p> 
$$ 
\begin{equation*}
   t_{cp,d,t} = 0
\end{equation*}
$$

$r_{cp}$ : 循環ポンプの変平均稼働率(-)  
$t_{cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間(h/h)  
$t_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの稼働時間(h/h)  

 循環ポンプの変平均稼働率（暖房運転と循環ポンプの稼働が同時に発生しないことを考慮するための係数）$r_{cp}$ 

<table>
    <caption>表4 循環ポンプの消費電力$P_{cp}$</caption>
    <thead style="background-color:lightgrey" >
        <tr>
            <th style="text-align:center; text-align:center; border:1px solid black;" rowspan=2 >地域区分</th>
            <th style="text-align:center; text-align:center; border:1px solid black;" colspan=5>年間日射量地域区分</th>
        </tr>
        <tr>
            <th style="text-align:center; text-align:center; border:1px solid black;">A1</th>
            <th style="text-align:center; text-align:center; border:1px solid black;">A2</th>
            <th style="text-align:center; text-align:center; border:1px solid black;">A3</th>
            <th style="text-align:center; text-align:center; border:1px solid black;">A4</th>
            <th style="text-align:center; text-align:center; border:1px solid black;">A5</th>
        </tr>
    </thead>
    <tbody>    
        <tr>
            <td style="border:1px solid black; text-align:center;">1</td>
            <td style="border:1px solid black; text-align:center;">0.41</td>
            <td style="border:1px solid black; text-align:center;">0.33</td>
            <td style="border:1px solid black; text-align:center;">0.32</td>
            <td style="border:1px solid black; text-align:center;">-</td>
            <td style="border:1px solid black; text-align:center;">-</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">2</td>
            <td style="border:1px solid black; text-align:center;">0.37</td>
            <td style="border:1px solid black; text-align:center;">0.44</td>
            <td style="border:1px solid black; text-align:center;">0.40</td>
            <td style="border:1px solid black; text-align:center;">-</td>
            <td style="border:1px solid black; text-align:center;">-</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">3</td>
            <td style="border:1px solid black; text-align:center;">0.44</td>
            <td style="border:1px solid black; text-align:center;">0.40</td>
            <td style="border:1px solid black; text-align:center;">0.35</td>
            <td style="border:1px solid black; text-align:center;">0.36</td>
            <td style="border:1px solid black; text-align:center;">0.39</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">4</td>
            <td style="border:1px solid black; text-align:center;">0.40</td>
            <td style="border:1px solid black; text-align:center;">0.45</td>
            <td style="border:1px solid black; text-align:center;">0.45</td>
            <td style="border:1px solid black; text-align:center;">0.39</td>
            <td style="border:1px solid black; text-align:center;">0.41</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">5</td>
            <td style="border:1px solid black; text-align:center;">0.50</td>
            <td style="border:1px solid black; text-align:center;">0.50</td>
            <td style="border:1px solid black; text-align:center;">0.53</td>
            <td style="border:1px solid black; text-align:center;">0.50</td>
            <td style="border:1px solid black; text-align:center;">0.47</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">6</td>
            <td style="border:1px solid black; text-align:center;">-</td>
            <td style="border:1px solid black; text-align:center;">0.67</td>
            <td style="border:1px solid black; text-align:center;">0.64</td>
            <td style="border:1px solid black; text-align:center;">0.62</td>
            <td style="border:1px solid black; text-align:center;">0.61</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">7</td>
            <td style="border:1px solid black; text-align:center;">0.78</td>
            <td style="border:1px solid black; text-align:center;">0.78</td>
            <td style="border:1px solid black; text-align:center;">0.78</td>
            <td style="border:1px solid black; text-align:center;">0.71</td>
            <td style="border:1px solid black; text-align:center;">0.74</td>
        </tr>
        <tr>
            <td style="border:1px solid black; text-align:center;">8</td>
            <td style="border:1px solid black; text-align:center;">-</td>
            <td style="border:1px solid black; text-align:center;">-</td>
            <td style="border:1px solid black; text-align:center;">1.00</td>
            <td style="border:1px solid black; text-align:center;">1.00</td>
            <td style="border:1px solid black; text-align:center;">1.00</td>
        </tr>
    </tbody> 
<table>

In [11]:
def get_r_cp(region, region_solar):
    return {
        'Region1' : {'A1' : 0.41, 'A2' : 0.33, 'A3' : 0.32, 'A4' : None, 'A5' : None},
        'Region2' : {'A1' : 0.37, 'A2' : 0.44, 'A3' : 0.40, 'A4' : None, 'A5' : None},
        'Region3' : {'A1' : 0.44, 'A2' : 0.40, 'A3' : 0.35, 'A4' : 0.36, 'A5' : 0.39},
        'Region4' : {'A1' : 0.40, 'A2' : 0.45, 'A3' : 0.45, 'A4' : 0.39, 'A5' : 0.41},
        'Region5' : {'A1' : 0.50, 'A2' : 0.50, 'A3' : 0.53, 'A4' : 0.50, 'A5' : 0.47},
        'Region6' : {'A1' : None, 'A2' : 0.67, 'A3' : 0.64, 'A4' : 0.62, 'A5' : 0.61},
        'Region7' : {'A1' : 0.78, 'A2' : 0.78, 'A3' : 0.78, 'A4' : 0.71, 'A5' : 0.74},
        'Region8' : {'A1' : None, 'A2' : None, 'A3' : 1.00, 'A4' : 1.00, 'A5' : 1.00}
    }[region][region_solar]

In [12]:
def get_t_cp(t_fan, region, region_solar):
    if 0 < t_fan:
        r_cp = get_r_cp(region, region_solar)
        return 1 * r_cp
    elif t_fan == 0:
        return 0
    else:
        raise Exeption

#### 8.3　空気搬送ファンの風量

日付$d$の時刻$t$における1時間当たりの空気搬送ファンの風量$V_{fan,d,t}$

$$ 
\begin{equation*}
   V_{fan,d,t} = V_{fan,P0} \times t_{fan,d,t}
\end{equation*}
$$

$t_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの稼働時間(h/h)  
$V_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの風量(m<sup>3</sup>/h)  
$V_{fan,P0}$ : 空気搬送ファンの送風機特性曲線において機外静圧をゼロとした時の空気搬送ファンの風量(m<sup>3</sup>/h)  

In [13]:
def get_V_fan(V_fan_P0, t_fan):
    return V_fan_P0 * t_fan

### 9. 集熱部

#### 9.1　集熱量

日付$d$の時刻$t$における1時間当たりの集熱部における集熱量$Q_{col,d,t}$

$$ 
\begin{equation*}
   Q_{col,d,t} = \rho_{air} \times c_{P_{air}} \times V_{fan,d,t} \times \left( \theta_{col,opg,d,t} - \theta_{ex,d,t} \right) \times 10^{-3}
\end{equation*}
$$

$c_{P_{air}}$ : 空気の比熱(kJ/(kg$\cdot$K))  
$Q_{col,d,t}$ : 空日付$d$の時刻$t$における1時間当たりの集熱部における集熱量(MJ/h)  
$V_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの風量(m<sup>3</sup>/h)  
$\theta_{col,opg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度(℃)  
$\theta_{ex,d,t}$ : 日付$d$の時刻$t$における外気温度(℃)  
$\rho_{air}$ : 空気の密度(kg/m<sup>3</sup>)  

空気の密度$\rho_{air}$

$$ 
\begin{equation*}
   \rho_{air} = 1.2
\end{equation*}
$$

空気の比熱$c_{P_{air}}$

$$ 
\begin{equation*}
   c_{P_{air}} = 1.006
\end{equation*}
$$

In [14]:
def get_Q_col(V_fan, theta_col_opg, theta_ex):
    rho_air = 1.2    # kg/m3
    c_P_air = 1.006  # kJ/(kg K)
    return rho_air * c_P_air * V_fan * (theta_col_opg - theta_ex) *10**(-3)

#### 9.2　空気搬送ファン停止時の集熱部の出口における空気温度

日付$d$の時刻$t$における空気搬送ファン停止時の集熱部の出口における空気温度$\theta_{col,nonopg,d,t}$

$$ 
\begin{equation*}
   \theta_{col,nonopg,d,t} = \frac{\displaystyle \sum_{j=1}^{n} \left( \theta_{col,nonopg,j,d,t} \times V_{col,j,d,t} \right)}{\displaystyle \sum_{j=1}^{n} V_{col,j,d,t}}
\end{equation*}
$$

$n$ : 集熱器群の数(-)  
$V_{col,j,d,t}$ : 日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量(m<sup>3</sup>/h)  
$\theta_{col,nonopg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱部の出口における空気温度(℃)  
$\theta_{col,nonopg,j,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱器群$j$の出口における空気温度(℃)  

In [15]:
def get_theta_col_nonopg(n_col, theta_col_nonopg, V_col):
    return sum(theta_col_nonopg[j] * V_col[j] for j in range(0,n_col+1,1)) / sum(V_col[j] for j in range(0,n_col+1,1))

#### 9.3　空気搬送ファン稼働時の集熱部の出口における空気温度

日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度$\theta_{col,opg,d,t}$

$$ 
\begin{equation*}
   \theta_{col,opg,d,t} = \frac{\displaystyle \sum_{j=1}^{n} \left( \theta_{col,opg,j,d,t} \times V_{col,j,d,t} \right)}{\displaystyle \sum_{j=1}^{n} V_{col,j,d,t}}
\end{equation*}
$$

$n$ : 集熱器群の数(-)  
$V_{col,j,d,t}$ : 日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量(-)  
$\theta_{col,opg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度(℃)  
$\theta_{col,opg,j,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱器群$j$の出口における空気温度(℃)  

In [16]:
def get_theta_col_opg(n_col, theta_col_opg, V_col):
    return sum(theta_col_opg[j] * V_col[j] for j in range(0,n_col+1,1)) / sum(V_col[j] for j in range(0,n_col,1))

#### 9.4　空気搬送ファン停止時の集熱器群の出口における空気温度

日付$d$の時刻$t$における空気搬送ファン停止時の集熱器群$j$の出口における空気温度$\theta_{col,nonopg,j,d,t}$

$$ 
\begin{equation*}
   \theta_{col,nonopg,j,d,t} = \frac{d_{0,j}}{d_{1,j}} \times I_{s,j,d,t} + \theta_{ex,d,t}
\end{equation*}
$$

$d_{0,j}$ : 集熱器群$j$を構成する集熱器の集熱効率特性線図一次近似式の切片(-)  
$d_{1,j}$ : 集熱器群$j$を構成する集熱器の集熱効率特性線図一次近似式の傾き(W/(m<sup>2</sup>$\cdot$K))  
$I_{s,j,d,t}$ : 日付$d$の時刻$t$における集熱器群$j$の単位面積当たりの平均日射量(W/m<sup>2</sup>)  
$\theta_{ex,d,t}$ : 日付$d$の時刻$t$における外気温度(℃)  
$\theta_{col,nonopg,j,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱器群$j$の出口における空気温度(℃)  

In [17]:
def get_theta_col_nonopg_j(d_0, d_1, I_s, theta_ex):
    return (d_0[j] / d_1[j] * I_s[j] + theta_ex for j in range(0,n_col,1))

#### 9.5　空気搬送ファン稼働時の集熱器群の出口における空気温度

日付$d$の時刻$t$における空気搬送ファン稼働時の集熱器群$j$の出口における空気温度$\theta_{col,opg,j,d,t}$

$$ 
\begin{equation*}
   \theta_{col,opg,j,d,t} = \theta_{col,nonopg,j,d,t} + \left( \theta_{ex,d,t} - \theta_{col,nonopg,j,d,t} \right) \times e^{- \frac{U_{c,j} \times A_{j}}{c_{P_{air}} \times \rho_{air} \times V_{col,j,d,t} \div 3600 \times 10^{-3}}}
\end{equation*}
$$

$A_{j}$ : 集熱器群$j$の面積(m<sup>2</sup>)  
$c_{P_{air}}$ : 空気の比熱(kJ/(kg$\cdot$K))  
$U_{c,j}$ : 集熱器群$j$を構成する集熱器の集熱器総合熱損失係数(W/(m<sup>2</sup>$\cdot$K))  
$V_{col,j,d,t}$ : 日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量(m<sup>3</sup>/h)  
$\theta_{col,opg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度(℃)  
$\theta_{col,nonopg,j,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱器群$j$の出口における空気温度(℃)  
$\theta_{ex,d,t}$ : 日付$d$の時刻$t$における外気温度(℃)  
$\rho_{air}$ : 空気の密度(kg/m<sup>3</sup>)  

日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量$V_{col,j,d,t}$

$$ 
\begin{equation*}
   V_{col,j,d,t} = V_{fan,P0} \times \frac{A_{j}}{\displaystyle \sum_{j=1}^{n} A_{j}}
\end{equation*}
$$

$A_{j}$ : 集熱器群$j$の面積(m<sup>2</sup>)  
$n$ : 集熱器群の数(-)  
$V_{col,j,d,t}$ : 日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量(m<sup>3</sup>/h)  
$V_{fan,P0}$ : 空気搬送ファンの送風機特性曲線において機外静圧をゼロとした時の空気搬送ファンの風量(m<sup>3</sup>/h)  

In [18]:
def get_V_col():
    return V_fan_P0 * A / sum(A[j] for j in range(0,n_col,1)) 

集熱器群$j$を構成する集熱器の集熱器総合熱損失係数$U_{c,j}$

$$ 
\begin{equation*}
   U_{c,j} = -c_{P_{air}} \times \dot{m}_{fan,test,j} \times 10^{3} \times ln \left( 1 - \frac{1}{c_{P_{air}} \times \dot{m}_{fan,test,j} \times 10^{3}} \times d_{1,j}  \right)
\end{equation*}
$$

$c_{P_{air}}$ : 空気の比熱(kJ/(kg$\cdot$K))  
$d_{1,j}$ : 集熱器群$j$を構成する集熱器の集熱効率特性線図一次近似式の傾き(W/(m<sup>2</sup>$\cdot$K))  
$\dot{m}_{fan,test,j}$ : 集熱器群$j$を構成する集熱器の集熱性能試験時における単位面積当たりの空気の質量流量(kg/(s$\cdot$m<sup>2</sup>))  
$U_{c,j}$ : 集熱器群$j$を構成する集熱器の集熱器総合熱損失係数(W/(m<sup>2</sup>$\cdot$K))  